In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch import optim

In [2]:
base_dir = os.path.dirname(os.getcwd())
sys.path.append(base_dir)

from core.dataloader import CelebALoader, PokeLoader
from core.models import VAE, Discriminator, Generator
from core.models import modules
from core.engine import ConfigFile, NCTrainer
from core.metrics import *
data_dir = "../../tmp"

# ./Session

In [3]:
config = ConfigFile("")

In [4]:
config.set_session_name("sandbox_mix")
config.setup_session()

/home/shahine/neural-conditioner/utils/IOHandler.py:25: UserWarning: directory /home/shahine/neural-conditioner/bin/sandbox_mix already exists
  warnings.warn(f"directory {full_path} already exists")
/home/shahine/neural-conditioner/utils/IOHandler.py:25: UserWarning: directory /home/shahine/neural-conditioner/bin/sandbox_mix/chkpt already exists
  warnings.warn(f"directory {full_path} already exists")
/home/shahine/neural-conditioner/utils/IOHandler.py:25: UserWarning: directory /home/shahine/neural-conditioner/bin/sandbox_mix/runs already exists
  warnings.warn(f"directory {full_path} already exists")
/home/shahine/neural-conditioner/utils/IOHandler.py:25: UserWarning: directory /home/shahine/neural-conditioner/bin/sandbox_mix/scores already exists
  warnings.warn(f"directory {full_path} already exists")



# ./Dataloader

In [30]:
transform = transforms.Compose([transforms.Resize((64, 64)),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                                     std=(0.5, 0.5, 0.5))])

dataloader = PokeLoader(data_dir=os.path.join(data_dir, 'pokemon'), 
                          batch_size=32,
                          train_transform=transform,
                          val_transform=transform,
                          validation_split=0.05)
config.set_dataloader(dataloader)

In [5]:
transform = transforms.Compose([transforms.Resize((64, 64)), 
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                                     std=(0.5, 0.5, 0.5))])

sampler = torch.utils.data.SubsetRandomSampler(indices=np.random.choice(96000, size=96000))

dataloader = CelebALoader(data_dir=data_dir, 
                          batch_size=128,
                          train_transform=transform,
                          val_transform=transform,
                          sampler=sampler,
                          validation_split=0.3)
config.set_dataloader(dataloader)

__Mask Generator :__

In [6]:
masks_kwargs = {'size': (64, 64),
                'coverage': (0.1, 0.5)}

config.update_kwargs(masks=masks_kwargs)

# ./Model

__Generator :__

In [6]:
kwargs =  [{'stride': 1, 'padding': 0}] + 3 * [{}] + [{'kernel_size': 4, 'bn': False, 'relu': False}]

generator = Generator(latent_size=(100, 1, 1),
                      nb_filters=[512, 256, 128, 64, 3],
                      conv_kwargs=kwargs)
config.set_model(generator)

__VAE :__

In [9]:
# vae = VAE(input_size=(6, 64, 64), 
#           z_dim=100, 
#           enc_nf = [64, 128, 128], 
#           dec_nf = [256, 128, 128],
#           enc_kwargs = {'padding': 1},
#           dec_kwargs = {'kernel_size': 4},
#           out_kwargs = {'kernel_size': 4, 'relu': False, 'bn': False},
#           out_channels=3)
# config.set_model(vae)

__Discriminator :__

In [7]:
kwargs = [{'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': False}] + \
         3 * [{'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': True}]

disc_kwargs = {'input_size': (3, 64, 64),
              'nb_filters': [512, 256, 128, 128],
              'conv_kwargs': kwargs}
config.update_kwargs(discriminator=disc_kwargs)

# ./Training params

__Criterion :__

In [8]:
criterion = nn.BCELoss()
config.set_criterion(criterion)

__Optimizers:__

In [9]:
gen_optimizer = torch.optim.Adam(generator.parameters(), lr=2e-4, betas=(0.5, 0.999))
disc_optimizer = {'lr': 2e-4,
                  'betas': (0.5, 0.999)}

config.set_optimizer(gen_optimizer)
config.update_kwargs(disc_optimizer=disc_optimizer)

__LR Scheduler:__

In [34]:
scheduler = optim.lr_scheduler.ExponentialLR(optimizer=config.optimizer, gamma=0.999)

config.set_lr_scheduler(scheduler)

__Metrics:__

In [16]:
callables = [accuracy, precision, recall]
config.set_metrics(callables)

__Mixup:__

In [11]:
mixup_kwargs = {'alpha': 0.75}
config.update_kwargs(mixup=mixup_kwargs)

__Training scope :__

In [12]:
epoch = 128
config.set_epochs(epoch)

In [17]:
config.dump()

In [14]:
config

_session_name : sandbox_mix
_session_dir : /home/shahine/neural-conditioner/bin/sandbox_mix
_criterion : BCELoss()
_optimizer : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.5, 0.999)
    eps: 1e-08
    lr: 0.0002
    weight_decay: 0
)
_metrics : None
_dataloader : Dataset CelebA
    Number of datapoints: 162770
    Root location: ../../tmp
    Target type: ['attr']
    Split: train
_init_epoch : 0
_epochs : 128
_lr_scheduler : None
_seed : 73
_kwargs : {'discriminator': {'input_size': (3, 64, 64), 'nb_filters': [512, 256, 128, 128], 'conv_kwargs': [{'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': False}, {'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': True}, {'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': True}, {'kernel_size': 4, 'padding': 1, 'stride': 2, 'leak': 0.2, 'bn': True}]}, 'disc_optimizer': {'lr': 0.0002, 'betas': (0.5, 0.999)}, 'mixup': {'alpha': 0.75}}

In [46]:
!tree -d ../bin

../bin
├── sandbox_gans
│   ├── chkpt
│   ├── runs
│   └── scores
└── sandbox_gans_2
    ├── chkpt
    ├── runs
    └── scores

8 directories


In [11]:
class Generator(nn.Module):
    """DCGAN Generator G(z)"""

    def __init__(self, latent_dim=100):
        super(Generator, self).__init__()
        self.latent_size = (latent_dim, )

        # Project and reshape
        self.linear = nn.Sequential(
            nn.Linear(latent_dim, 512 * 4 * 4, bias=True),
            nn.BatchNorm1d(512 * 4 * 4),
            nn.ReLU(inplace=True))

        # Upsample
        self.features = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh())
        """
        self.features = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=5, stride=2, padding=2,
                output_padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, 5, 2, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 5, 2, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, 5, 2, 2, 1, bias=False),
            nn.Tanh())
        """

    def forward(self, x):
        x = self.linear(x).view(x.size(0), -1, 4, 4)
        return self.features(x)


In [12]:
generator = Generator()
generator(torch.randn((4, 100))).shape

torch.Size([4, 3, 64, 64])

In [13]:
generator.latent_size

(100,)

In [14]:
config.set_model(generator)